In [7]:
import torch
import pandas as pd
from torch import nn
from torch.utils.data import Dataset
from collections import Counter
from torchtext.vocab import vocab
from tqdm import tqdm



class W2VDataSet(Dataset) :
    def __init__(self,dir,index_col=0) -> None:
        super().__init__()
        self.data_load = pd.read_csv(dir,index_col=index_col)
        self.vocab = vocab()
        self.counter = Counter()
    def hello(self) : 
        pass
    
    # def __getitem__() :
    #     pass
    # def __call__(self, ) :
    #     pass


In [10]:
list_book_intro = W2VDataSet('./data/bookraw_6-1.csv').data_load['book_intro']
list_book_intro =list_book_intro.fillna(0)
[sen.split('.') for sen in list_book_intro if sen != 0]

In [58]:
import re
asdasd = W2VDataSet('./data/bookraw_6-1.csv').data_load

test = asdasd.iloc[1,1]

def findHan(text: str) -> str:
    return re.findall("[\u3130-\u318F\uAC00-\uD7A3]+", text)
    # return re.sub("[^\u3130-\u318F\uAC00-\uD7A3]+",'' ,text)

# print(test)
fi_a = findHan(test)

In [78]:
isbn = pd.read_csv('./data/rawBookInfo.csv')

isbn['col1']

0       9791158393038
1       9791158393359
2       9788970936208
3       9791162245552
4       9791163033554
            ...      
5276    9791192265384
5277    9788956749167
5278    9791186710821
5279    9791191600889
5280    9791165921521
Name: col1, Length: 5281, dtype: int64

In [168]:
import requests
from bs4 import BeautifulSoup
import re

ISBN = 9791158393038
kyoboUrl = f"http://www.kyobobook.co.kr/product/detailViewKor.laf?ejkGb=KOR&mallGb=KOR&barcode={ISBN}"
kyoboHtml = requests.get(kyoboUrl)
kyoboSoup = BeautifulSoup(kyoboHtml.content, "html.parser")

book_title: str = kyoboSoup.find(class_="prod_title").string

# 도서 toc 추출, <br/>을 가지고 목차를 구분하므로 split 사용
book_toc: str = kyoboSoup.find(class_='book_contents_item')
book_toc = str(book_toc).split('<br/>')[1:-1]


# 도서 소개 추출, <br/>을 가지고 목차를 구분하므로 split 사용, tag제거
book_intro: str = kyoboSoup.find_all('div','info_text')[-1]
book_intro_list = str(book_intro).split('<br/>')

book_intro = []
for i in book_intro_list :
    book_intro += re.sub('<.*>','',i).split('.')


publisher: str = kyoboSoup.find(class_='book_publish_review')

if publisher is not None : 
    publisher = publisher.p.get_text()
    publisher = publisher.split('.')
else :
    publisher = []

result = [book_title]

for chars in [book_toc,book_intro,publisher] :
    # ''제거
    chars = list(filter(None,chars))

    # 한국어 영어 제외하고 모두 제거
    chars = [re.sub('[^A-Za-z\u3130-\u318F\uAC00-\uD7A3]',' ' ,i) for i in chars]

    # 공백 하나로 줄이기 ex) '  ' -> ' '
    chars = [re.sub(' +',' ',i).strip(' ') for i in chars]

    result.append(chars)




,0,1,2,3
0,그림과 실습으로 배우는 도커 & 쿠버네티스,"[도커란 무엇인가, 안개 속에 숨겨진 도커의 정체는, 데이터나 프로그램을 독립된 환...",[이 책은 컨테이너 기술이 어렵게 느껴지는 엔지니어나 백엔드 기술에 자신이 없는 분...,[]


,0,1,2
0,1,2,3
